In [1]:
# You don't need these two lines if you have pip installed topologicpy
import sys
sys.path.append("C:/Users/wassimj/Documents/GitHub/")

## Import the needed classes

In [1]:
# Import the needed classes
from topologicpy.Vertex import Vertex
from topologicpy.Face import Face
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Color import Color
from topologicpy.Plotly import Plotly
from topologicpy.EnergyModel import EnergyModel

## Set the input variables for path names and load geometries

In [6]:
import os
# Set the needed file paths. Change to match your computer paths
weatherFilePath = "./topologicpy/assets/EnergyModel/GBR_London.Gatwick.037760_IWEC.epw"
designDayFilePath = "./topologicpy/assets/EnergyModel/GBR_London.Gatwick.037760_IWEC.ddy"
osBinaryPath = "D:/openstudioapplication-1.5.0/bin/openstudio.exe"
outputFolder ="./topologicpy/assets/EnergyModel/"
buildingPath = "./topologicpy/assets/EnergyModel/TopologicBuilding.brep"
shadingPath = "./topologicpy/assets/EnergyModel/TopologicShadingSurfaces.brep"

building = Topology.ByImportedBRep(buildingPath)
shadingSurfaces = Topology.ByImportedBRep(shadingPath)


# Create aperuters (External Windows)

In [3]:
# Create apertures (windows) by scaling the external vertical walls

# Specify a glazing (window to wall) ratio:
wwr = 0.5

# Decompose the building
d = CellComplex.Decompose(building)

# Get the external vertical faces
walls = d['externalVerticalFaces']
apertures = []
for wall in walls:
    centroid = Topology.Centroid(wall)
    aperture = Topology.Scale(wall, centroid, wwr, wwr, wwr)
    apertures.append(aperture)
building = Topology.AddApertures(building, apertures, subTopologyType="face")

d = CellComplex.Decompose(building)
bhf = d['bottomHorizontalFaces']
ihf = d['internalHorizontalFaces']
thf = d ['topHorizontalFaces']
hf = bhf+ihf+thf
print(len(hf))
floorLevels = [Vertex.Z(Topology.Centroid(f)) for f in hf]

30


## Create an Energy Model

In [4]:
model = EnergyModel.ByTopology(building=building, shadingSurfaces=shadingSurfaces, glazingRatio=0.25)

## Export the model to OSM and GBXML

In [ ]:
# Uncomment these files and change output path if you wish to export the model
#status01 = EnergyModel.ExportToOSM(model, outputFolder+"/EnergModel.osm", overwrite=True)
#status02 = EnergyModel.ExportToGbXML(model, outputFolder+"/EnergModel.xml", overwrite=True)


## Run the Simulation

In [7]:
# Run the simulation
model = EnergyModel.Run(model, weatherFilePath=weatherFilePath,
                        osBinaryPath=osBinaryPath, outputFolder=outputFolder)


Running Simulation:   0%|          | 0/100 [00:00<?, ?it/s]

## Get the space names used in the energy model

In [8]:
# Get the topologies from the energy model. These have the correct dictionaries with space names etc.
topologies = EnergyModel.Topologies(model)
cells = topologies['cells']

# Get the required row names for the SQL query
names = []
for cell in cells:
    d = Topology.Dictionary(cell)
    name = Dictionary.ValueAtKey(d, 'TOPOLOGIC_name')
    name = name.upper()+"_THERMAL_ZONE"
    names.append(name)

## Set the report, table, and column names to retrieve from the SQL data

In [9]:
# Set the report, table, and column names to retrieve from the SQL data

# Here we specify three possible reports. We get them by index for convenience.
reports = ["HVACSizingSummary", "HVACSizingSummary", "SystemSummary"]
forStrings = ["Entire Facility", "Entire Facility", "Entire Facility"]
tableNames = ["Zone Sensible Cooling", "Zone Sensible Heating", "Time Not Comfortable Based on Simple ASHRAE 55-2004"]
columnNames = ["User Design Load", "User Design Load", "Summer or Winter Clothes"]
unitsList = ["W", "W", "hr"]

# Choose report 0, 1, or 2
index = 0 

reportName = reports[index]
reportForString = forStrings[index]
tableName = tableNames[index]
columnName = columnNames[index]
units = unitsList[index]

# Get the values from the SQL data
values = EnergyModel.Query(model,
                          reportName = reportName,
                          reportForString = reportForString,
                          tableName = tableName,
                          columnName = columnName,
                          rowNames = names,
                          units = units)


## Apply the values to the bottom face(s) of each space/cell.

In [11]:
# This section is to apply the values only to the bottom face of each cell.
faces = []
selectors = []
for i, cell in enumerate(cells):
    d = Cell.Decompose(cell)
    bottomFaces = d['bottomHorizontalFaces']
    for bottomFace in bottomFaces:
        d = Dictionary.ByKeysValues(["value", "label"], [values[i], str(round(values[i], 2))])
        faces.append(bottomFace)
        s = Face.InternalVertex(bottomFace)
        s = Topology.SetDictionary(s, d)
        selectors.append(s)
faceCluster = Cluster.ByTopologies(faces)
faceCluster = Topology.TransferDictionariesBySelectors(faceCluster, selectors, tranFaces=True, tolerance=0.0001)

TypeError: type NoneType doesn't define __round__ method

## Create a Plotly figure and draw it.

In [13]:
# Create a Plotly figure and draw it.
# Create a cluster to represent the building envelope (translucent). This is just for visualisation.
envelopeCluster = Cluster.ByTopologies(cells+apertures+Topology.Faces(shadingSurfaces))
plotlyData = Plotly.DataByTopology(envelopeCluster, faceOpacity=0.1)

# Add the color-coded faces
colorScale = "thermal"
#plotlyData += Plotly.DataByTopology(faceCluster, faceOpacity=1, faceGroupKey="value",
#                              faceLabelKey="label", faceGroups=values,
#                              colorScale=colorScale)
# Create a Plotly Figure
figure = Plotly.FigureByData(plotlyData, width=800, height=600)

# Add a colorbar
figure = Plotly.AddColorBar(figure, values=values, width=30, outlineWidth=1,
                            nTicks=len(values), title=tableName, subTitle=columnName,
                            units=units, colorScale=colorScale, mantissa=2)
# Show plotly figure
Plotly.Show(figure)

TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'